HW6 Мельчук А.Б.

# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [2]:
!pip install implicit

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420110 sha256=b572523968aa45f95ba099b34e4ec10847d6cc0827d1f87e85606e55786d47e7
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [45]:
data = pd.read_csv('./data/retail_train.csv')
item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)




In [46]:
n_items_before = data['item_id'].nunique()

data = prefilter_items(data, item_features=item_features, take_n_popular=5000)

n_items_after = data['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 89051 to 5001


In [47]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
7,2375,26984851516,1,1085983,1,2.99,364,-0.4,1642,1,0.0,0.0,2.99
11,1364,26984896261,1,999999,1,2.19,31742,0.0,1520,1,0.0,0.0,2.19


In [48]:
recommender = MainRecommender(data_train_lvl_1)

In [49]:
recommender.get_als_recommendations(2375, N=200)

[899624,
 871756,
 1046545,
 1103691,
 1044078,
 993838,
 999779,
 925862,
 12731432,
 1102207,
 9835223,
 872062,
 847066,
 854852,
 1051323,
 12352293,
 12731517,
 1009770,
 1042942,
 1022827,
 12301839,
 9836106,
 999714,
 1107420,
 12731544,
 1094833,
 1033046,
 844179,
 1106523,
 1096573,
 1038663,
 973181,
 949965,
 965267,
 1084613,
 1079067,
 8090521,
 1000753,
 855626,
 841220,
 832678,
 998556,
 863885,
 5585510,
 827919,
 947858,
 1075470,
 830304,
 865456,
 1081177,
 902172,
 982393,
 835300,
 871061,
 826666,
 887428,
 869322,
 1004906,
 1138467,
 5566717,
 850925,
 1097458,
 1001702,
 5568721,
 952563,
 823990,
 913278,
 870547,
 12757140,
 1000736,
 887325,
 1096728,
 12262978,
 967205,
 5569845,
 6534480,
 5592094,
 1024319,
 8090537,
 1137284,
 9832469,
 965766,
 829138,
 944534,
 844462,
 944486,
 1134678,
 1074333,
 883932,
 1100972,
 1092937,
 1014948,
 850139,
 12810391,
 973135,
 1025535,
 913026,
 988835,
 865233,
 828106,
 6904459,
 8358065,
 1090931,
 952163,
 

In [50]:
recommender.get_own_recommendations(2375, N=200)

[948640,
 918046,
 847962,
 1079023,
 907099,
 873980,
 872062,
 884694,
 10285454,
 1107760,
 979674,
 5567582,
 10308345,
 1069531,
 950935,
 1015474,
 1102207,
 847066,
 1020770,
 9521787,
 974265,
 940996,
 8019845,
 5567194,
 12811490,
 1003616,
 973181,
 855914,
 890719,
 982955,
 9677152,
 1072685,
 998519,
 1131382,
 1021715,
 12263119,
 960791,
 1113261,
 7441873,
 986021,
 15926887,
 1038692,
 9677748,
 9297223,
 1120991,
 927030,
 6391532,
 12757653,
 1046919,
 989069,
 1068451,
 951954,
 835300,
 937343,
 1047249,
 13876348,
 1061732,
 981601,
 1121028,
 1087547,
 828393,
 996269,
 951951,
 1036093,
 1023815,
 5570408,
 827667,
 1082454,
 1006878,
 5570048,
 841309,
 1078652,
 1115553,
 1056492,
 1138467,
 1004945,
 947858,
 1092885,
 1121694,
 938138,
 8019916,
 827919,
 984315,
 10341855,
 883932,
 8291322,
 1096794,
 1028938,
 1087618,
 8020166,
 1082185,
 866871,
 930666,
 825994,
 823990,
 910151,
 848029,
 896613,
 12301839,
 1117219,
 1135258,
 1068957,
 1046545,
 89

In [51]:
recommender.get_similar_items_recommendation(2375, N=200)

[1046545,
 937292,
 1044078,
 12781666,
 907099,
 896613,
 1005274,
 885863,
 1047619,
 999999,
 1127831,
 1040807,
 9524291,
 1070702,
 1027090,
 1025535,
 9835606,
 899624,
 12301100,
 903350,
 871611,
 863447,
 878996,
 12810436,
 906923,
 866871,
 910032,
 828106,
 925694,
 6633210,
 960732,
 848029,
 941883,
 999858,
 12132277,
 1001656,
 954651,
 989069,
 995896,
 6979253,
 1090247,
 5568378,
 1011457,
 5591170,
 836645,
 843558,
 941883,
 12757653,
 869322,
 916122,
 12301109,
 12811490,
 12523757,
 9419444,
 9653714,
 1004857,
 846502,
 12988031,
 1008032,
 968992,
 9803545,
 909811,
 8065410,
 13654702,
 1112238,
 999999,
 1074754,
 907631,
 9553397,
 933637,
 9707240,
 825343,
 1131488,
 10285022,
 925862,
 1119942,
 878996,
 850562,
 861279,
 828106,
 826666,
 12301100,
 865456,
 885863,
 969945,
 869728,
 968732,
 957411,
 1105488,
 12262778,
 969941,
 8090537,
 1018740,
 9836106,
 860167,
 5569792,
 899459,
 1038998,
 1120559,
 9553397,
 937292,
 948650,
 5569230,
 882247,

In [52]:
recommender.get_similar_users_recommendation(2375, N=200)

[12523928,
 820612,
 10457044,
 902640,
 1102003,
 854133,
 949257,
 9553048,
 12427353,
 1026945,
 857130,
 1089568,
 12696089,
 5568758,
 921406,
 1107760,
 1043534,
 916990,
 837495,
 10198378,
 977780,
 1012801,
 873044,
 5569135,
 985699,
 6979437,
 1129805,
 1055403,
 921438,
 928749,
 983665,
 1117219,
 847434,
 1123045,
 873324,
 973016,
 10198378,
 8019902,
 5569309,
 1118120,
 13213202,
 1128477,
 7168129,
 896757,
 915064,
 9392700,
 892573,
 988697,
 1082212,
 1124971,
 825665,
 918598,
 959455,
 874563,
 916990,
 925350,
 974265,
 979674,
 823099,
 1018818,
 908312,
 908217,
 9831557,
 822970,
 820486,
 1114811,
 902377,
 8020166,
 1134483,
 5570590,
 8119101,
 1053022,
 868645,
 12484491,
 1076580,
 7443137,
 1057168,
 1101502,
 997025,
 7443137,
 1096573,
 12352054,
 823031,
 1111986,
 851188,
 948640,
 894360,
 979674,
 1082990,
 5707857,
 882826,
 1072693,
 1048299,
 865718,
 854133,
 6979753,
 1112205,
 1075765,
 913249,
 1047525,
 873324,
 1065017,
 948239,
 897671,


### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 200 кандидатов (k=200)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [53]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[865456, 999999, 878285, 922281, 940947, 98389..."
1,2,"[999999, 839656, 866211, 893867, 1011457, 1103..."


In [54]:
# your_code
train_users = data_train_lvl_1['user_id'].unique()
result_lvl_1 = result_lvl_1[result_lvl_1['user_id'].isin(train_users)]

result_lvl_1['candidates_own'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))
result_lvl_1['candidates_als'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
result_lvl_1['candidates_sim_items'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=200))
# result_lvl_1['candidates_sim_users'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=200))
result_lvl_1

,user_id,actual,candidates_own,candidates_als,candidates_sim_items
0,1,"[865456, 999999, 878285, 922281, 940947, 98389...","[856942, 9297615, 5577022, 9655212, 888104, 10...","[5577022, 12352201, 9655212, 856942, 825123, 1...","[824758, 826597, 9297615, 5577022, 920200, 983..."
1,2,"[999999, 839656, 866211, 893867, 1011457, 1103...","[911974, 1076580, 5567582, 1103898, 1056620, 9...","[5569230, 908064, 1051211, 1021324, 972665, 97...","[8090537, 1044078, 5569845, 819978, 880888, 98..."
2,4,"[883932, 970760, 1035676, 999999, 831063, 8914...","[6391541, 1052294, 891423, 936470, 1137010, 83...","[821741, 891423, 6391541, 880888, 7441873, 109...","[1090507, 903526, 960075, 1111786, 910032, 847..."
3,6,"[1024306, 6548453, 1098844, 999999, 8357613, 9...","[13003092, 972416, 995598, 923600, 1138596, 10...","[1082185, 878996, 857006, 1127179, 965267, 108...","[948650, 5569845, 823990, 878996, 999999, 1105..."
4,7,"[999999, 993638, 1106523, 5572738, 909714, 102...","[998519, 894360, 7147142, 9338009, 896666, 939...","[1100140, 839419, 878290, 10285022, 970605, 10...","[1029743, 7147145, 1044078, 936685, 6602697, 9..."
...,...,...,...,...,...
2095,2496,"[831509, 999999, 820321, 839243, 865456, 98239...","[872826, 983665, 991546, 1134296, 12452939, 74...","[844179, 916122, 1044078, 1004906, 899624, 999...","[985999, 5569845, 1041796, 1070702, 12301109, ..."
2096,2497,"[824759, 999999, 965430, 970202, 977867, 98128...","[870515, 1102207, 1117219, 1057168, 1135834, 1...","[899624, 9527184, 5569230, 963971, 957951, 809...","[1043301, 865026, 5569471, 850102, 910473, 826..."
2097,2498,"[865511, 962991, 999999, 5564901, 844991, 9407...","[1022066, 1076580, 1100379, 5565356, 931579, 8...","[12326050, 997796, 1004385, 957839, 834382, 71...","[1070820, 1022097, 1041796, 7410071, 829722, 8..."
2098,2499,"[999999, 882308, 865992, 869322, 899624, 90412...","[7168055, 1128395, 6904613, 5570048, 889989, 8...","[1029743, 962991, 5569230, 907631, 1068719, 10...","[1022097, 5568995, 7442008, 851394, 12301109, ..."


In [55]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_own'], row['actual'], 200), axis=1).mean()

0.3678429844225468

In [56]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_als'], row['actual'], 200), axis=1).mean()

0.2874284838212492

In [57]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_sim_items'], row['actual'], 200), axis=1).mean()

0.31351868694851687

In [58]:
# result_lvl_1.apply(lambda row: recall_at_k(row['candidates_sim_users'], row['actual'], 200), axis=1).mean()

In [59]:
result_lvl_1['candidates_own_20'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=20))
result_lvl_1['candidates_own_50'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_lvl_1['candidates_own_100'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=100))
result_lvl_1['candidates_own_500'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))

In [60]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_own_20'], row['actual'], 20), axis=1).mean()

0.11105115040192286

In [61]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_own_50'], row['actual'], 50), axis=1).mean()

0.18172520630192224

In [62]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_own_100'], row['actual'], 100), axis=1).mean()

0.2658796946273117

In [63]:
result_lvl_1.apply(lambda row: recall_at_k(row['candidates_own_500'], row['actual'], 500), axis=1).mean()

0.47673186593969946

Рекоммендации собственных товаров и схожик с ними дают значительно лучший результат. Качество модели растёт вместе с К.

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [64]:
# your_code

In [65]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']


train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))
data_train_lvl_2.head(4)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
2107471,2021,40618753059,594,896862,2,5.00,443,-2.98,101,86,0.0,0.0,2.50
2107473,2021,40618753059,594,1019142,2,5.00,443,-1.98,101,86,0.0,0.0,2.50
2107476,2021,40618753059,594,9835223,1,9.27,443,-3.63,101,86,0.0,0.0,9.27
2108010,1753,40618809138,594,999999,1,29.99,345,0.00,8,86,0.0,0.0,29.99


In [66]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['drop'] = 1  

users_lvl_2.head(4)

,user_id,item_id,drop
0,2021,950935,1
0,2021,1119454,1
0,2021,835578,1
0,2021,863762,1


In [67]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('drop', axis=1, inplace=True)

In [68]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(data_train_lvl_2, on=['user_id', 'item_id'], how='left')


targets_lvl_2.head(20)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2021,950935,0.0,2193,GROCERY,National,FRZN NOVELTIES/WTR ICE,WATER ICE,24 CT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,1119454,0.0,910,GROCERY,National,BAKED BREAD/BUNS/ROLLS,HAMBURGER BUNS,8 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,835578,0.0,539,DRUG GM,National,CIGARETTES,CIGARETTES,971267 PK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,863762,0.0,6046,DRUG GM,National,PREPAID WIRELESS&ACCESSORIES,PREPAID WIRELESS CARDS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,1019142,1.0,1007,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - FLAVORED/OTHER,1 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.061875e+10,594.0,2.0,5.0,443.0,-1.98,101.0,86.0,0.0,0.0,2.5
5,2021,1029125,0.0,136,GROCERY,National,DOG FOODS,DOG BISCUITS (HARD BISCUIT),24 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021,1097398,0.0,111,DRUG GM,National,CIGARETTES,CIGARETTES,CTN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021,870515,0.0,2,PRODUCE,National,ONIONS,ONIONS RED (BULK&BAG),40 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021,6424460,0.0,69,PASTRY,Private,PIES,PIES: FRUIT/NUT,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021,988277,0.0,253,NUTRITION,National,FROZEN,FROZEN BREAKFAST,9 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
user_features

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16
...,...,...,...,...,...,...,...,...
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498


In [72]:
test = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
targets_lvl_2.head(20)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2021,950935,0.0,2193,GROCERY,National,FRZN NOVELTIES/WTR ICE,WATER ICE,24 CT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,1119454,0.0,910,GROCERY,National,BAKED BREAD/BUNS/ROLLS,HAMBURGER BUNS,8 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,835578,0.0,539,DRUG GM,National,CIGARETTES,CIGARETTES,971267 PK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,863762,0.0,6046,DRUG GM,National,PREPAID WIRELESS&ACCESSORIES,PREPAID WIRELESS CARDS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,1019142,1.0,1007,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - FLAVORED/OTHER,1 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.061875e+10,594.0,2.0,5.0,443.0,-1.98,101.0,86.0,0.0,0.0,2.5
5,2021,1029125,0.0,136,GROCERY,National,DOG FOODS,DOG BISCUITS (HARD BISCUIT),24 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021,1097398,0.0,111,DRUG GM,National,CIGARETTES,CIGARETTES,CTN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021,870515,0.0,2,PRODUCE,National,ONIONS,ONIONS RED (BULK&BAG),40 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021,6424460,0.0,69,PASTRY,Private,PIES,PIES: FRUIT/NUT,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021,988277,0.0,253,NUTRITION,National,FROZEN,FROZEN BREAKFAST,9 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
